#Dependencies

In [ ]:
%cd drive/MyDrive/sqlovaa/sqlova

/content/drive/MyDrive/sqlovaa/sqlova


In [ ]:
!pip install flask-ngrok

In [ ]:
!pip3 install sqlalchemy==1.3
!pip3 install records==0.5.2
!pip3 install babel matplotlib defusedxml tqdm
!pip3 install ujson stanza

import stanza
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
#stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir
#!ls $CORENLP_HOME

!pip3 install konlpy
!pip3 install random

ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [ ]:
# KoBERT requirements

!pip install mxnet>=1.4.0
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyrebase 3.0.27 requires requests==2.11.1, but you have requests 2.26.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-2nu7k83v
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-2nu7k83v


In [ ]:
!pip3 install pyrebase

  Using cached requests-2.11.1-py2.py3-none-any.whl (514 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.0.1 requires requests>=2.19.0, but you have requests 2.11.1 which is incompatible.
tensorboard 2.6.0 requires requests<3,>=2.21.0, but you have requests 2.11.1 which is incompatible.
spacy 2.2.4 requires requests<3.0.0,>=2.13.0, but you have requests 2.11.1 which is incompatible.
pandas-datareader 0.9.0 requires requests>=2.19.0, but you have requests 2.11.1 which is incompatible.
mxnet 1.8.0.post0 requires requests<3,>=2.20.0, but you have requests 2.11.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.11.1 which is incompatibl

In [ ]:
!pip3 install pyrebase

###mcS

In [ ]:
""import os, json

from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template

import pyrebase

import add_question_ale
from add_question_ale import question_to_json
import add_csv_ale
from add_csv_ale import csv_to_sqlite,csv_to_json
import annotate_ws_okt
from annotate_ws_okt import annotate_ale, count_lines
from predict_ale import run_predict, args_ale
#from predict_ale_kobert_imp import run_predict

from tqdm import tqdm

import jsonl

import uuid

config = {
    
}

firebase = pyrebase.initialize_app(config)

db = firebase.database()
storage = firebase.storage()

app = Flask(__name__)
run_with_ngrok(app)   

user_id=""

dsaved = './data_and_model/flask_test_1019'

def annotate(din, dout, user_id, att):
    fquestions = os.path.join(din, user_id) + '_'+att + '.jsonl'
    ftable = os.path.join(din, user_id) + '.tables.jsonl'
    ftok = os.path.join(dout, user_id) + '_'+att + '_tok.jsonl'
    
    with open(fquestions, encoding='utf-8') as fq, open(ftable, encoding='utf-8') as ft, open(ftok, 'wt', encoding='utf-8') as fo:
            
            tables = {}
            for line in tqdm(ft, total=count_lines(ftable)):
                d = json.loads(line)
                tables[d['id']] = d
            print('loading examples')
            n_written = 0
            cnt = -1
            for line in tqdm(fq, total=count_lines(fquestions)):
                cnt += 1
                d = json.loads(line)
                a = annotate_ale(d, tables[d['table_id']])
                fo.write(json.dumps(a, ensure_ascii=False) + '\n')
                n_written += 1

def print_predicted(dsaved, user_id,att):
    origin_data = jsonl.load_jsonl("{}/results_{}_{}.jsonl".format(dsaved,user_id,att))
    nlu_sql={}
    for d in origin_data:
        nlu_sql[d['nlu']+'\n']=d['sql']+'\n'
    return nlu_sql


# route main page
@app.route('/')
def index():
    return render_template("index3.html")

# data prediction
@app.route('/add_data', methods=['POST'])
#@app.route('/', methods=['POST'])
def add_data():
    if request.method =='POST':
        #upload data
        #user_id = request.form['userid']
        #user_id = "tts"
        user_id = uuid.uuid4()
        #tid = request.form['tid']
        json_file_name = '{}.jsonl'.format(user_id)

        table = request.files['table']
        table_name = table.filename
        tid = table_name[:-4]
        table.save(os.path.join(dsaved, table_name))
        
        """query = request.form['query']
        questions = query.split('\n')

        for q in questions:
          print(q)
          question_to_json(tid, q.strip('\n\r'), json_file_name, dsaved)

        #storage.child("{}/{}".format(user_id, json_file_name)).put("{}/{}".format(dsaved, json_file_name))

        #storage.child("tables/{}.csv".format(tid)).put(table)
        """
        csv_to_sqlite(tid, table_name, '{}.db'.format(user_id), dsaved)
        csv_to_json(tid, table_name, '{}.tables.jsonl'.format(user_id), dsaved)
        #storage.child("{}/{}.db".format(user_id, user_id)).put("{}/{}.db".format(dsaved, user_id))
        #storage.child("{}/{}.tables.jsonl".format(user_id, user_id)).put("{}/{}.tables.jsonl".format(dsaved, user_id))
        #os.remove(os.path.join(dsaved, table_name))

        return render_template("query.html", tid=tid, user_id = user_id, att = 0)
    return render_template("index3.html")

@app.route('/predict', methods=['POST'])
def predict():
    if request.method =='POST':
        #upload data
        user_id = request.form['uid']
        att = request.form['att']
        tid = request.form['tid']
        json_file_name = '{}_{}.jsonl'.format(user_id, att)

        nlu = request.form['nlu']
        questions = nlu.split('\n')

        for q in questions:
          print(q)
          question_to_json(tid, q.strip('\n\r'), json_file_name, dsaved)


        annotate(dsaved, dsaved, user_id, att)
        #storage.child("{}/{}_tok.jsonl".format(user_id, user_id)).put("{}/{}_tok.jsonl".format(dsaved, user_id))
        #return render_template("index3.html", predicted = predicted)
        
        run_predict('../result_files/okt/mcS/model_best.pt', '../result_files/okt/mcS/model_bert_best.pt', './data_and_model', dsaved, user_id, att,'./data_and_model/flask_test_1019/')
        result_dict = print_predicted(dsaved, user_id, att)

        return render_template("query.html", predicted=result_dict.items(), tid=tid, user_id = user_id)
    return render_template("index3.html")

if __name__=='__main__':
	app.run()""

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0f50-35-221-196-106.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Oct/2021 20:36:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:36:52] "GET /static/css/style0.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:36:52] "GET /static/img/dog.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:36:52] "GET /static/js/TableCsv.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:36:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Oct/2021 20:36:59] "POST /add_data HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:37:00] "GET /static/css/style1.css HTTP/1.1" 200 -


캐나다 방영일이 2008년 2월 11일인 경우 시리즈 번호는 무엇인가요?
캐나다 방영일이 2008년 2월 11일인 경우 시리즈 번호는 무엇인가요?


100%|██████████| 1/1 [00:00<00:00, 3744.91it/s]


loading examples


100%|██████████| 1/1 [00:00<00:00, 24.14it/s]
[2021-10-18 20:37:05,001] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-8-2b611b9bc12e>", line 165, in predic

###KoBERT

In [8]:
import os, json

from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template

import pyrebase

import add_question_ale
from add_question_ale import question_to_json
import add_csv_ale
from add_csv_ale import csv_to_sqlite,csv_to_json
import annotate_ws_okt
from annotate_ws_okt import annotate_ale, count_lines
from predict_ale_kobert_imp import run_predict, args_ale

from tqdm import tqdm

import jsonl

import uuid

config = {
    
}

firebase = pyrebase.initialize_app(config)

db = firebase.database()
storage = firebase.storage()

app = Flask(__name__)
run_with_ngrok(app)   

user_id=""

dsaved = './data_and_model/flask_test_1019'

def annotate(din, dout, user_id, att):
    fquestions = os.path.join(din, user_id) + '_'+att + '.jsonl'
    ftable = os.path.join(din, user_id) + '.tables.jsonl'
    ftok = os.path.join(dout, user_id) + '_'+att + '_tok.jsonl'
    
    with open(fquestions, encoding='utf-8') as fq, open(ftable, encoding='utf-8') as ft, open(ftok, 'wt', encoding='utf-8') as fo:
            
            tables = {}
            for line in tqdm(ft, total=count_lines(ftable)):
                d = json.loads(line)
                tables[d['id']] = d
            print('loading examples')
            n_written = 0
            cnt = -1
            for line in tqdm(fq, total=count_lines(fquestions)):
                cnt += 1
                d = json.loads(line)
                a = annotate_ale(d, tables[d['table_id']])
                fo.write(json.dumps(a, ensure_ascii=False) + '\n')
                n_written += 1

def print_predicted(dsaved, user_id,att):
    origin_data = jsonl.load_jsonl("{}/results_{}_{}.jsonl".format(dsaved,user_id,att))
    nlu_sql={}
    for d in origin_data:
        nlu_sql[d['nlu']+'\n']=d['sql']+'\n'
    return nlu_sql


# route main page
@app.route('/')
def index():
    return render_template("index3.html")

# data prediction
@app.route('/add_data', methods=['POST'])
#@app.route('/', methods=['POST'])
def add_data():
    if request.method =='POST':
        #upload data
        #user_id = request.form['userid']
        #user_id = "tts"
        user_id = uuid.uuid4()
        #tid = request.form['tid']
        json_file_name = '{}.jsonl'.format(user_id)

        table = request.files['table']
        table_name = table.filename
        tid = table_name[:-4]
        table.save(os.path.join(dsaved, table_name))
        
        """query = request.form['query']
        questions = query.split('\n')

        for q in questions:
          print(q)
          question_to_json(tid, q.strip('\n\r'), json_file_name, dsaved)

        #storage.child("{}/{}".format(user_id, json_file_name)).put("{}/{}".format(dsaved, json_file_name))

        #storage.child("tables/{}.csv".format(tid)).put(table)
        """
        csv_to_sqlite(tid, table_name, '{}.db'.format(user_id), dsaved)
        csv_to_json(tid, table_name, '{}.tables.jsonl'.format(user_id), dsaved)
        #storage.child("{}/{}.db".format(user_id, user_id)).put("{}/{}.db".format(dsaved, user_id))
        #storage.child("{}/{}.tables.jsonl".format(user_id, user_id)).put("{}/{}.tables.jsonl".format(dsaved, user_id))
        #os.remove(os.path.join(dsaved, table_name))

        return render_template("query.html", tid=tid, user_id = user_id, att = 0)
    return render_template("index3.html")

@app.route('/predict', methods=['POST'])
def predict():
    if request.method =='POST':
        #upload data
        user_id = request.form['uid']
        att = request.form['att']
        tid = request.form['tid']
        json_file_name = '{}_{}.jsonl'.format(user_id, att)

        nlu = request.form['nlu']
        questions = nlu.split('\n')

        for q in questions:
          print(q)
          question_to_json(tid, q.strip('\n\r'), json_file_name, dsaved)


        annotate(dsaved, dsaved, user_id, att)
        #storage.child("{}/{}_tok.jsonl".format(user_id, user_id)).put("{}/{}_tok.jsonl".format(dsaved, user_id))
        #return render_template("index3.html", predicted = predicted)
        
        #run_predict('../result_files/okt/mcS/model_best.pt', '../result_files/okt/mcS/model_bert_best.pt', './data_and_model', dsaved, user_id, att,'./data_and_model/flask_test_1019/')
        run_predict('../result_files/okt/KoBERT/model_best.pt', '../result_files/okt/KoBERT/model_bert_best.pt', './data_and_model', dsaved, user_id, att,'./data_and_model/flask_test_1019/')
        result_dict = print_predicted(dsaved, user_id, att)

        return render_template("query.html", predicted=result_dict.items(), tid=tid, user_id = user_id)
    return render_template("index3.html")

if __name__=='__main__':
	app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a875-35-190-140-197.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Oct/2021 21:55:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 21:55:08] "GET /static/css/style0.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 21:55:08] "GET /static/img/dog.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 21:55:08] "GET /static/js/TableCsv.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 21:55:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Oct/2021 21:55:16] "POST /add_data HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 21:55:16] "GET /static/css/style1.css HTTP/1.1" 200 -


2008년 4월 4일 미국에서 방영된 에피소드의 이름은 무엇이었습니까?
2008년 4월 4일 미국에서 방영된 에피소드의 이름은 무엇이었습니까?


...

100%|██████████| 5/5 [00:00<00:00, 98.79it/s]

BERT-type: kobert
Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 8002
hidden_size: 768
num_hidden_layer: 12
num_attention_heads: 12
hidden_act: gelu
intermediate_size: 3072
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02


using cached model
using cached model
using cached model
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.0001


127.0.0.1 - - [18/Oct/2021 21:56:19] "POST /predict HTTP/1.1" 200 -


{"query": {"agg": 0, "sel": 0, "conds": [[3, 0, "2008년 3월 17일"]]}, "table_id": "k22222", "nlu": "2008년 3월 17일의 캐나다 방영일에는 시즌 번호가 몇 개 있습니까?", "sql": "SELECT (시즌 번호) FROM k22222 WHERE 캐나다 방영일 = 2008년 3월 17일"}

1
{"query": {"agg": 0, "sel": 0, "conds": [[2, 0, "&quot;믿는 걸 멈추지 마&quot;"]]}, "table_id": "k22222", "nlu": "&quot;믿는 걸 멈추지 마&quot;라는 제목의 에피소드가 시즌 중 가장 높은 수를 기록한 에피소드는 무엇입니까?", "sql": "SELECT (시즌 번호) FROM k22222 WHERE 제목 = &quot;믿는 걸 멈추지 마&quot;"}

2
{"query": {"agg": 0, "sel": 3, "conds": [[4, 0, "2008년 8월 8일"]]}, "table_id": "k22222", "nlu": "미국 방영일이 2008년 8월 8일 일 떄의 캐나다 방영일이 무엇이었습니까?", "sql": "SELECT (캐나다 방영일) FROM k22222 WHERE 미국 방영일 = 2008년 8월 8일"}

3
{"query": {"agg": 0, "sel": 1, "conds": [[3, 0, "2008년 2월 11일"]]}, "table_id": "k22222", "nlu": "캐나다 방영일이 2008년 2월 11일인 경우 시리즈 번호는 무엇인가요?", "sql": "SELECT (시리즈 번호) FROM k22222 WHERE 캐나다 방영일 = 2008년 2월 11일"}

4
{"query": {"agg": 0, "sel": 5, "conds": [[4, 0, "2008년 4월 4일"]]}, "table_id": "k22222", "nlu": "2008년 4월 4일의 미국 방영일의 생산 코드